### setup

In [1]:
import gzip
import json
from typing import List
from collections import Counter, defaultdict
from tqdm import tqdm
import os
import inspect_ai
from openai import OpenAI
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login
from dotenv import load_dotenv
import torch as t
import subprocess
import contextlib
import shutil
import ast
import textwrap

In [2]:
from inspect_ai import Task, task
from inspect_ai.dataset import Sample, hf_dataset
from inspect_ai.util import ExecResult, sandbox
from inspect_ai.scorer import CORRECT, INCORRECT, Score, Scorer, Target, accuracy, scorer, stderr
from inspect_ai.solver import TaskState, generate
from inspect_ai.model import get_model
from inspect_ai.log import read_eval_log
import re

In [3]:
load_dotenv()
login(token = os.environ['HF_TOKEN'])

KeyError: 'HF_TOKEN'

In [ ]:
IMPORT_HELPER = {
    "python": [
        "import math",
        "import re",
        "import sys",
        "import copy",
        "import datetime",
        "import itertools",
        "import collections",
        "import heapq",
        "import statistics",
        "import functools",
        "import hashlib",
        "import numpy",
        "import numpy as np",
        "import string",
        "from typing import *",
        "from collections import *",
    ],
    "go"    : [
        "math",
        "strings",
        "fmt",
        "strconv",
        "time",
        "bytes",
        "regexp",
        "sort",
        "math/rand",
        "crypto/md5",
    ],
    "cpp"   : [
        "#include<stdlib.h>",
        "#include<algorithm>",
        "#include<math.h>",
        "#include<stdio.h>",
        "#include<vector>",
        "#include<string>",
        "#include<climits>",
        "#include<cstring>",
        "#include<iostream>",
    ],
}

# instruction prepended to code problem
HUMANEVAL_INSTRUCTION = """
Read the following function signature and docstring, and fully implement
the function described. Your response should only contain the code for
this function.\n
"""

LANG_PREFIX = {
    "cpp"          : "// language: C++",
    "java"         : "// language: Java",
    "js"           : "// language: JavaScript",
    "javascript"   : "// language: JavaScript",
    "go"           : "// language: Go",
    "python"       : "# language: Python",
}

In [ ]:
# model = get_model(
#         'hf/meta-llama/Llama-3.1-8B-Instruct', 
#         device = 'auto',
#         torch_dtype=t.bfloat16,
# )

### Data Processing

In [4]:
def stream_jsonl_all(filename: str):
    results = []
    fp = gzip.open(open(filename, "rb"), "rt")
    for line in fp:
        if any(not x.isspace() for x in line):
            results.append(json.loads(line))
    fp.close()

    return results

In [5]:
python_content = stream_jsonl_all('data/python_data.gz')
cpp_content = stream_jsonl_all('data/cpp_data.gz')
go_content = stream_jsonl_all('data/go_data.gz')
java_content = stream_jsonl_all('data/java_data.gz')
js_content = stream_jsonl_all('data/js_data.gz')
content = [python_content, cpp_content, go_content, java_content, js_content]

In [6]:
generations = stream_jsonl_all('data/python_generations.gz')
generations[0]['generation']

'    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = abs(elem - elem2)\n                if distance < threshold:\n                    return True\n\n    return False\n'

In [7]:
for lang in content:
    print(lang[0].keys())
    print()

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'import', 'docstring', 'declaration', 'canonical_solution', 'test', 'test_setup', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'text', 'declaration', 'example_test'])

dict_keys(['task_id', 'prompt', 'canonical_solution', 'test', 'declaration', 'example_test'])



### LLM output => code  
*I need to see LLM output & transform that into code*

#### Running Eval

In [8]:
# FIND CODE

def find_code_new(completion: str) -> str:
    pattern_1 = re.compile(r"```python\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if matches:
        extracted_answer = matches[0]
        extracted_answer = extract_function_body_new(extracted_answer)
    else:
        extracted_answer = completion
    return str(extracted_answer)


def extract_function_body_new(code: str) -> str:
    try:
        tree = ast.parse(code)
        for node in tree.body:
            if not isinstance(node, ast.FunctionDef):
                continue
            code_lines = code.splitlines()
            start = node.body[0].lineno - 1
            end = node.body[-1].end_lineno
            body_lines = code_lines[start:end]
            return "\n".join(body_lines)

    except Exception as e:
        print(f"Error extracting function body: {e}")
        return "errormsg"

In [ ]:
def get_final_python(state, completion):
    imports = "\n".join(IMPORT_HELPER["python"]) + "\n"
    final_code = imports + state.metadata['prompt'] + completion + "\n" + state.metadata['test'] + "\n"
    return final_code

In [ ]:
@scorer(metrics=[accuracy(), stderr()])
def python_scorer() -> Scorer:
    async def score(state: TaskState, target: Target) -> Score:
        idx = state.sample_id
        model_completion = state.output.completion
        processed_completion = find_code_new(model_completion)
        final_code = get_final_python(state, processed_completion)

        if 'errormsg' in processed_completion:
            print(f'error in sample: {idx}')

        try:
            result = await sandbox().exec(
                cmd=["python", "-c", final_code],
                timeout=30,
            )
        except TimeoutError:
            result = ExecResult(False, 1, "", "Verification timed out.")

        return Score(
            value=CORRECT if result.success else INCORRECT,
            explanation="".join(
                ["The following verification code was executed:\n\n"]
                + [final_code]
                + [f"\n\nThe submission was incorrect\n\n{result.stderr}"]
                if not result.success
                else [""]
            ),
            metadata={
                'completion': model_completion,
                'processed': processed_completion,
                'final_code': final_code,
            }
        )

    return score


In [ ]:
lang = 'python'

def humaneval_record_to_sample(record):
    model_input = HUMANEVAL_INSTRUCTION + LANG_PREFIX[lang] + '\n' + record['prompt'] 
    
    return Sample(
        id=record["task_id"],
        input=model_input,
        target=record["canonical_solution"],
        metadata={
            "prompt": record["prompt"],
            "test": record["test"],
        },
    )

humaneval_dataset = hf_dataset(
    path = 'THUDM/humaneval-x',
    name = lang,
    split = 'test',
    sample_fields = humaneval_record_to_sample,
    trust = True,
)

In [ ]:
samples = 164

@task
def humaneval():
    return Task(
        dataset = humaneval_dataset[:samples],
        solver = generate(),
        scorer = python_scorer(),
        sandbox = 'local',
    )

In [ ]:
epochs = 1
inspect_ai.eval(humaneval(), model = model, epochs = epochs)

#### Checking outputs

In [35]:
def find_code_test(completion: str) -> str:
    # lang = 'python'

    pattern_1 = re.compile(r"```(?:python|javascript|java|cpp|go)\n(.*?)```", re.DOTALL)
    pattern_2 = re.compile(r"```\n(.*?)```", re.DOTALL)
    matches = pattern_1.findall(completion) + pattern_2.findall(completion)
    if matches:
        extracted_answer = matches[0]
    else:
        extracted_answer = completion
    return extracted_answer

In [36]:
idx = 0

log = read_eval_log('/root/srf-project/test_humaneval-x/logs/python/python_llama-8b_1.eval')
model_out = log.samples[idx].messages[-1].content

print(model_out)

```python
from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """
    Check if in given list of numbers, are any two numbers closer to each other than given threshold.

    Args:
        numbers (List[float]): A list of floating point numbers.
        threshold (float): The minimum distance between two numbers to be considered close.

    Returns:
        bool: True if any two numbers are closer than the threshold, False otherwise.
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False
```


In [39]:
out = find_code_test(model_out)
print(out)

from typing import List

def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """
    Check if in given list of numbers, are any two numbers closer to each other than given threshold.

    Args:
        numbers (List[float]): A list of floating point numbers.
        threshold (float): The minimum distance between two numbers to be considered close.

    Returns:
        bool: True if any two numbers are closer than the threshold, False otherwise.
    """
    for i in range(len(numbers)):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) < threshold:
                return True
    return False



In [ ]:
target_id = 145
idx = 0
model_out = ''

for i, sample in enumerate(log.samples):
    id = int(sample.id.split('/')[-1])
    if id == target_id:
        idx = i
        model_out = sample.messages[-1].content
        print(i)
        print()
        print(model_out)
        break


In [ ]:
print(find_code_new(model_out))